In [1]:
import chess.pgn
import pandas as pd
import chess
import numpy as np
from scipy import linalg

In [2]:
ls -shr

total 89M
 20K README.md                              148K chess-values.png
 89M lichess_db_standard_rated_2013-01.pgn  120K chess-and-eigenvectors.ipynb


In [3]:
board = chess.Board()

In [4]:
pgn = open('lichess_db_standard_rated_2013-01.pgn',encoding="utf-8")

In [5]:
#creates a list with one entry for each space on the board
spaces = []
for i in range(1,9):
    for j in range (1,9):
        spaces.append(chr(j+96)+str(i))

In [6]:
#create a matrix to tally weights for every pair of opposing pieces
pieces_list = [space for space in spaces if any(map(space.__contains__, ['1','2','7','8']))]
weights = pd.DataFrame(0,index=pieces_list, columns=pieces_list)

In [7]:
#create a state dictionary of each square and the initial square of each piece
#Those 'None' entries are the empty squares in the middle of the board, before the first move
#Each piece is on it's own square to begin
state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}
#state_dict

In [8]:
'''game = chess.pgn.read_game(pgn)

while game is not None:
    if game.headers["BlackElo"] != "?":
        print('black',game.headers["BlackElo"])
        if int(game.headers["BlackElo"]) > 1800 and int(game.headers["WhiteElo"]) > 1800:
            print('black',game.headers["BlackElo"])
            print('white',game.headers["WhiteElo"])

        game = chess.pgn.read_game(pgn)
'''

'game = chess.pgn.read_game(pgn)\n\nwhile game is not None:\n    if game.headers["BlackElo"] != "?":\n        print(\'black\',game.headers["BlackElo"])\n        if int(game.headers["BlackElo"]) > 1800 and int(game.headers["WhiteElo"]) > 1800:\n            print(\'black\',game.headers["BlackElo"])\n            print(\'white\',game.headers["WhiteElo"])\n\n        game = chess.pgn.read_game(pgn)\n'

In [9]:
game_counter = 0
total_move_counter = 0
game = chess.pgn.read_game(pgn)

while game is not None:
    if game.headers["BlackElo"] != "?" and game.headers["WhiteElo"] != "?":

        if int(game.headers["BlackElo"]) > 1800 and int(game.headers["WhiteElo"]) > 1800:

            counter = 0
            state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

            #print(game)
            #do stuff here



            #for each line in the file
            for move in game.mainline_moves():
                counter += 1
                if board.is_pseudo_legal(move):


                    if board.is_capture(move):
                        capturing_piece = state_dict[chess.square_name(move.from_square)]
                        captured_piece = state_dict[chess.square_name(move.to_square)]
                        #print('capturing piece',capturing_piece)
                        #print('captured piece',captured_piece)

                        if board.is_en_passant(move):
                            #do en passant stuff here
                            #print("en passant")


                            #print ('to square',chess.square_name(move.to_square))
                            #print ('to square file',chess.square_file(move.to_square))
                            #print ('to square rank',chess.square_rank(move.to_square))
                            file = chess.square_file(move.to_square)
                            rank = chess.square_rank(move.to_square)

                            if board.turn:
                                #white's turn, take piece behind square where pawn ends up, decrease rank by 1
                                rank = rank-1
                                #print('white')
                            else:
                                #blacks's turn, take piece behind square where pawn ends up
                                rank = rank+1
                                #print("black")
                            take_square = chess.square_name(chess.square(file, rank)) 
                            #print('take square',take_square)

                            #increment an entry in the weights matrix when one piece takes another
                            weights.loc[state_dict[chess.square_name(move.from_square)],state_dict[take_square]] += 1

                            #update the take square to be empty
                            state_dict[take_square] = None

                            #update the piece on the target square
                            state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                            #update the source square to be empty
                            state_dict[chess.square_name(move.from_square)] = None

                        elif capturing_piece is not None:
                            if captured_piece is not None:

                                #record the kill in a matrix of all kills
                                #print(counter,chess.square_name(move.from_square), "takes on",chess.square_name(move.to_square))
                                #print(counter,state_dict[chess.square_name(move.from_square)], "takes",state_dict[chess.square_name(move.to_square)])

                                #what piece is there now?


                                #increment an entry in the weights matrix when one piece takes another
                                weights.loc[state_dict[chess.square_name(move.from_square)],state_dict[chess.square_name(move.to_square)]] += 1

                                #update the piece on the target square
                                state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                                #update the source square to be empty
                                state_dict[chess.square_name(move.from_square)] = None

                                #board.push(move)  # Make the move


                                #if df.loc[i+1,'move_no'] == 1:
                                #    state_dict = {space:(space  if any(map(space.__contains__, ['1','2','7','8'])) else None) for space  in spaces}

                    else:


                        #add castling special cases to record both moves
                        if board.is_kingside_castling(move):
                            if board.turn:
                                #print(counter,'... and white castles kingside')
                                #move the appopriate rook
                                state_dict['f1'] = 'h1'
                                #leaving the rook source square empty
                                state_dict['h1'] = 'None'
                                #board.push(move)  # Make the move



                            elif not board.turn:
                                #print(counter,'... and black castles kingside')
                                #move the appopriate rook
                                state_dict['f8'] = 'h8'
                                #leaving the rook source square empty
                                state_dict['h8'] = 'None'
                                #board.push(move)  # Make the move

                        elif board.is_queenside_castling(move):
                            if board.turn:
                                #print(counter,'... and white castles queenside')
                                #move the appopriate rook
                                state_dict['d1'] = 'a1'
                                #leaving the rook source square empty
                                state_dict['a1'] = 'None'
                                #board.push(move)  # Make the move

                            elif not board.turn:
                                #print(counter,'... and black castles queenside')
                                #move the appopriate rook
                                state_dict['d8'] = 'a8'
                                #leaving the rook source square empty
                                state_dict['a8'] = 'None'
                                #board.push(move)  # Make the move

                        else:
                            #update the piece on the target square
                            state_dict[chess.square_name(move.to_square)] = state_dict[chess.square_name(move.from_square)]

                            #update the source square to be empty
                            state_dict[chess.square_name(move.from_square)] = None


                            #print(counter, chess.square_name(move.from_square),chess.square_name(move.to_square))

                board.push(move)  # Make the move
                #print(state_dict)

            if board.is_checkmate():
                checkers_list = [state_dict[chess.square_name(x)] for x in list(board.checkers())]
                if checkers_list[0] is not None:
                    piece_giving_check = str(checkers_list[0])

                    if board.turn:
                        captured_king = 'e1'
                    else:
                        captured_king = 'e8'
                    weights.loc[piece_giving_check,captured_king] += 1


            game_counter += 1
            #print('end of game',game_counter)

            total_move_counter += counter
            #print('total moves analyzed',total_move_counter)


            #print('board outcome',board.outcome())


            #print('is game over?',board.is_game_over())
            board.reset()

    game = chess.pgn.read_game(pgn)




    #print(state_dict)

    

In [10]:
print(game_counter,'games analyzed')
print(total_move_counter,'moves analyzed')

9458 games analyzed
720888 moves analyzed


In [11]:
weights

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,g7,h7,a8,b8,c8,d8,e8,f8,g8,h8
a1,0,5,5,9,0,6,4,7,20,13,...,227,238,1414,363,375,517,127,361,286,1124
b1,1,0,1,0,0,2,2,1,6,6,...,137,116,248,525,577,305,27,704,1261,240
c1,19,7,0,18,0,6,5,17,12,19,...,213,338,211,787,0,286,17,1990,1575,313
d1,9,5,7,0,0,4,4,9,9,14,...,478,476,649,969,1285,2361,473,803,884,649
e1,0,0,0,0,0,0,0,0,0,0,...,19,26,48,38,43,112,0,65,54,56
f1,8,0,2,5,0,0,2,7,14,14,...,161,269,270,1316,1741,262,19,0,998,245
g1,6,2,7,2,0,2,0,11,10,6,...,214,168,184,1580,726,274,16,514,490,273
h1,15,4,6,3,0,5,5,0,23,17,...,313,298,1023,376,474,537,131,402,426,1311
a2,1,1,2,0,0,2,1,1,0,6,...,12,11,48,123,84,75,28,40,30,18
b2,1,4,0,0,0,1,3,3,17,0,...,11,5,46,119,65,68,23,677,303,18


In [12]:
#This still clearly contains some bug or bugs, not sure where the self-captures could be coming from. Possibly a
#mistake in the state dict when accounting for en passants.
#The number of mistakes is still very low, apparently less than one percent, which doesn't skew results all that much.

In [13]:
#subtracting the identity matrix is necessary, forming an implicit set of homogeneous equations
a = weights - np.identity(32)
#print(a)

In [14]:
b = np.zeros(32)
#print(b)

In [15]:
x = linalg.solve(a, b)
x

array([ 0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,
        0.,  0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0., -0.,
        0.,  0., -0., -0., -0.,  0.])

### Why doesn't it work?
### Because the solver is finding the trivial solution. i.e. just a vector full of zeroes.
### To check that there is actually a non trivial solution, the determinant of A must be non-zero.

In [16]:
np.linalg.det(a)

-1.1916276450784986e+86

### OK since the determinant is not zero, there must be at least one non-trivial solution. How to find it?
### An homogenous linear equation can be solved by finding the eigenvectors.

In [17]:
eigen_values, eigen_vectors = np.linalg.eig(a)  
eigen_vectors

array([[ 0.22327441+0.j        , -0.2207312 +0.j        ,
         0.37096329+0.j        , ...,  0.03527276+0.0009305j ,
        -0.00326098+0.j        ,  0.00622057+0.j        ],
       [ 0.18854767+0.j        , -0.19091971+0.j        ,
        -0.0265757 +0.j        , ..., -0.00438824+0.00594435j,
        -0.00609356+0.j        , -0.00557457+0.j        ],
       [ 0.23213479+0.j        , -0.2272675 +0.j        ,
         0.03496255+0.j        , ..., -0.02637562-0.00814739j,
        -0.041075  +0.j        , -0.05260237+0.j        ],
       ...,
       [ 0.23123072+0.j        ,  0.23087415+0.j        ,
         0.00930437+0.j        , ...,  0.01744266-0.03237866j,
        -0.066381  +0.j        ,  0.04262479+0.j        ],
       [ 0.19492967+0.j        ,  0.19527372+0.j        ,
        -0.03993506+0.j        , ..., -0.00955405+0.03515199j,
         0.00976645+0.j        , -0.01058679+0.j        ],
       [ 0.22239426+0.j        ,  0.21937265+0.j        ,
         0.36554273+0.j       

In [18]:
values = eigen_vectors[:, np.argmax(eigen_values)] 
values
#Notice that the entries of the eigenvector corresponding to the largest eigenvalue are all real valued,
#So it's OK to throw away the imaginary component

array([0.22327441+0.j, 0.18854767+0.j, 0.23213479+0.j, 0.43569064+0.j,
       0.02225811+0.j, 0.22179421+0.j, 0.20526125+0.j, 0.22652632+0.j,
       0.03347074+0.j, 0.07025364+0.j, 0.07444251+0.j, 0.06900105+0.j,
       0.08060852+0.j, 0.07045885+0.j, 0.05630167+0.j, 0.03735422+0.j,
       0.03045405+0.j, 0.0699491 +0.j, 0.07650185+0.j, 0.07416608+0.j,
       0.0915428 +0.j, 0.06861211+0.j, 0.05463455+0.j, 0.03961383+0.j,
       0.22244249+0.j, 0.1973809 +0.j, 0.21954016+0.j, 0.42654404+0.j,
       0.03236502+0.j, 0.23123072+0.j, 0.19492967+0.j, 0.22239426+0.j])

In [19]:
## Let's normalize the values


In [20]:
list = []
for i in values:
    list.append(abs(float(i)))

<ipython-input-20-fd238d3eb4fa>:3: ComplexWarning: Casting complex values to real discards the imaginary part
  list.append(abs(float(i)))


In [21]:
#The fractional inverse of the average of all the pawns
factor = 1 / np.mean(list[8:24])

In [22]:
normed = [i*factor for i in list]
#normed

In [23]:
normalized_list = ["%.2f"%i for i in normed]

In [24]:
#tempo difference
white_sum = sum(normed[0:16])
black_sum = sum(normed[16:32])


print('total value of white pieces is', white_sum)
print('total value of black pieces is', black_sum)
print('total difference in value between white and black pieces is', white_sum-black_sum)
print('This is equal to the point value of the tempo advantage for white')

total value of white pieces is 36.05303643805179
total value of black pieces is 36.132012955710664
total difference in value between white and black pieces is -0.07897651765887304
This is equal to the point value of the tempo advantage for white


In [25]:
print(normalized_list[24:32])
print(normalized_list[16:24])
print(normalized_list[8:16])
print(normalized_list[0:8])

['3.57', '3.17', '3.52', '6.84', '0.52', '3.71', '3.13', '3.57']
['0.49', '1.12', '1.23', '1.19', '1.47', '1.10', '0.88', '0.64']
['0.54', '1.13', '1.19', '1.11', '1.29', '1.13', '0.90', '0.60']
['3.58', '3.02', '3.72', '6.99', '0.36', '3.56', '3.29', '3.63']


In [26]:
if not os.path.isfile('blank-board.png'):
    svg2png(bytestring=chess.svg.board(board),write_to='blank-board.png',scale=2)

NameError: name 'os' is not defined

In [ ]:
im = cv2.imread('blank-board.png')

In [ ]:
black_major_pieces = ' '.join(normalized_list[24:32])
black_pawns = ' '.join(normalized_list[16:24])
white_pawns = ' '.join(normalized_list[8:16])
white_major_pieces = ' '.join(normalized_list[0:8])

In [ ]:
color = (0, 0, 255)
black = (0, 0, 0)
white = (255, 255, 255)

cv2.putText(img=im, 
            text=black_major_pieces, org=(30, 132), 
            fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1.05, 
            color=white,thickness=2)
cv2.putText(img=im, 
            text=black_pawns, org=(30, 210), 
            fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1.05, 
            color=white,thickness=2)
cv2.putText(img=im, 
            text=white_pawns, org=(30, 584), 
            fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1.05, 
            color=black,thickness=2)
cv2.putText(img=im, 
            text=white_major_pieces, org=(30, 674), 
            fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1.05, 
            color=black,thickness=2)


In [ ]:
fig, ax = plt.subplots(dpi=200)
plt.imshow(im,)

In [ ]:
cv2.imwrite('chess-values.png',im)

In [ ]:
#Notice that, for each pair of opposing pawns, the black pawn is stronger in 7 out of 8 pairs
#Why should this be? Possibly because white has the initial tempo for attacking early in the game,
#giving the black pawns many opportunities for capturing pieces defensively, and practically necessitating it.

In [ ]:
#find average strength of all the pieces taken by each piece
#eg, rook strength seems low in general, probably because they come out later in the game
#and have fewer opportunities to capture lots of pieces,
#but they probably capture, on average, stronger pieces, in proportion to their own strength 